# UAS_processing
Creators: Seth Ackerman (@sackerman-usgs), Emily Sturdivant (@esturdivant-usgs)

Jupyter notebook to process image and GPX files.


## Detailed workflow:

1. Read the GPX file, which is a telemetry log file produced by the 3DR Solo in tlog format and converted to GPX in Mission Planner. View a dataframe from the GPX file.
2. Parse the time field in the GPX dataframe. Add fields datetime_utc and epoch_utc.
3. Export the dataframe as a table in CSV format and map the flight path from the GPX navigation data.
4. Plot the flight path on an aerial photo basemap.
5. Initialize a dataframe for the images. Include the original filename and the time in UTC, Epoch, and ISO formats.
6. Export a CSV of the dataframe. Plot the image times and the GPX elevations by time to check that they match.
7. Rename the photos using the survey number, the flight and camera ID, the time in ISO format, and the original filename.
8. Geotag the photos from the GPX file using the Geosync tool in ExifTool.
9. Update the EXIF tags to standard values.

## Requirements
Python 3 with modules (in addition to defaults):

- lxml
- PIL, which includes ExifTool by Phil Harvey
- pandas
- numpy
- matplotlib
- ipyleaflet

These are satisfied by using the `IOOS3` environment in Anaconda.

### To use ipyleaflet

```
conda install -c conda-forge ipyleaflet
```


```
pip install ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet
```


## Inputs/outputs

Variables:

- homedir: working directory that contains image folder and tlog folder (with gpx file) and where outputs will be saved.
- flight: flight ID that matches the image folder name and the gpx file, usually in format fX
- logfile: gpx file path
- imagefolder: image folder path

Output products:

- CSV of pertinent telemetry data converted from GPX
- PNG of flight path created from GPX
- new folder of images with standardized filenames and image headers populated with contextual information




### Import packages and define the namespace for the GPX schema

If you get an error here, you will have to add packages to your Python distribution

In [ ]:
import os, string, copy, sys, subprocess
import shutil
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lxml import etree
from PIL import Image, ExifTags
import ipyleaflet as ipyl

# set pandas output to limit to N records
pd.set_option('max_rows', 8)

%matplotlib inline

## Input Variables 

In [ ]:
# Set local filepaths:
homedir = r'/Users/esturdivant/Desktop/photos_test/ricoh' # '/Data/2018_PlumIsland/2018_015_FA_PlumIs_Feb/test_script/ricoh'
logfile = r'/Users/esturdivant/Desktop/photos_test/ricoh/f1.gpx'
imagefolder = r'/Users/esturdivant/Desktop/photos_test/ricoh/f1'

# Mission info - would be nice for logfile and image folder names to correspond to naming convention, as below
fan = '2018-015-FA' # input('Field activity number (e.g. "2017-010-FA"): ')
flight_id = 'f06' 
cam_id = 'r01' 

# Automated file/folder names
navcsvoutfile = os.path.splitext(logfile)[0]+'_gpx.csv'
imgoutdir = imagefolder+'_new'

# WHSC EXIF population
credit = "U.S. Geological Survey"
comment = """Low-altitude aerial photograph of Plum Island, MA from 
survey {0} (https://cmgds.marine.usgs.gov/fan_info.php?fa={0}).""".format(fan)
keywords = "Plum Island, Massachusetts, {}, UAS, nadir, USGS".format(fan)
artist = "WHCMSC AIM Group"
contact = "WHSC_data_contact@usgs.gov"

In [ ]:
# ALTERNATIVE: prompt for user input

# homedir = input("Which directory contains your raw images and telemetry data (e.g. {})? ".format(homedir))
# flightdirname = input("Which flight? Must match folder name in tlogs folder (e.g. f6). ")
# logfile = os.path.join(homedir,'tlogs', flightdirname+'.gpx')
# imagefolder = os.path.join(homedir,'images', flightdirname)
# fan = input('Field activity number (e.g. "2017-010-FA"): ')
# flight_id = input('Flight number (e.g. "f04"): ') 
# cam_id = input('Camera code (e.g. "r01"): ')

### Verify the user input data looks right before continuing

In [ ]:
print("""Input nav file: {}\nOutput nav file: {}\nWorking JPEG dir: {}\nProcessed JPEG dir: {}""".format(
    logfile, navcsvoutfile, imagefolder, imgoutdir))
input_check = "FA = {}    flight = {}    sensor = {}".format(fan, flight_id, cam_id)
print ("input data: "+ input_check)

while True:
   answer = input('Does the input data above look right?:')
   if answer.lower().startswith("y"):
        break
   elif answer.lower().startswith("n"):
      print("ok, try to re-run the script and enter the correct FA, flight and sensor")
      sys.exit()

if not os.path.isfile(logfile):
    print("We don't see the specified file (logfile variable): {}".format(logfile))
    sys.exit()
else:
    print("Great!! Looks like the logfile is present as well. Carry on.")

## Parse GPX file and extract components into dataframe

In [ ]:
def gpx_tag_to_pdseries(tree, namespace, tag):
    elist = tree.xpath('./def:trk//def:trkpt//def:'+tag, namespaces=namespace)
    ser = pd.Series([e.text for e in elist], name=tag)
    return(ser)

# Parse GPX
tree = etree.parse(logfile)

# Extract latitude and longitude to initialize GPX dataframe
namespace = {'def': 'http://www.topografix.com/GPX/1/1'}
elist = tree.xpath('./def:trk//def:trkpt',namespaces=namespace)
gpxdf = pd.DataFrame([e.values() for e in elist], columns=['lat', 'lon']).apply(pd.to_numeric)

# Extract each tag (including time) and add to dataframe
taglist = ['time', 'ele', 'ele2', 'course', 'roll', 'pitch', 'mode']
for tag in taglist:
    gpxdf = gpxdf.join(pd.to_numeric(gpx_tag_to_pdseries(tree, namespace, tag), errors='ignore'))

# Check number of records and number of unique times 
print ("Number of records in GPX file: ", len(gpxdf.index))
print ("Number of unique time stamps in GPX file: {}\n".format(len(gpxdf.time.unique())))

gpxdf.describe()
gpxdf

### Add datetime field to gpxdf dataframe

Convert the time field to: 

- datetime_utc: datetime in UTC
- epoch_utc: Unix epoch time, which counts seconds from January 1, 1970

In [ ]:
# Convert values in 'time' field to datetime UTC and convert datetime UTC to Epoch UTC
tfmt_gpx = '%Y-%m-%dT%H:%M:%S' #e.g. 2017-05-04T14:14:12-04:00
gpxdf['datetime_utc'] = pd.to_datetime(gpxdf['time'], format = tfmt_gpx)
gpxdf['epoch_utc'] = gpxdf['datetime_utc'].astype(np.int64) // 10**9

## this applies the local time zone offset to create a UTC epochtime value
## March 2018 - commented out the tzoffset part for Plum Island survey - collected in UTC
# gpxdf['epoch_utc'] = gpxdf['epoch_utc'].astype(str).astype(int) 

gpxdf_uniquetimes = gpxdf.drop_duplicates(subset='time')
print("Number of records in unique times GPX file: ", len(gpxdf_uniquetimes.index))

gpxdf

### Export (csv) and plot (png) the GPX data

We're not sure how the interpolation works. Does simply '-' tell it to plot as a line, which includes interpolation? 

In [ ]:
# Export CSV
gpxdf.to_csv(navcsvoutfile, index=False)
print ("Exported output CSV file to:", navcsvoutfile,"\n")

# Plot the flight path
fig = plt.figure()
plt.plot(gpxdf.lon,gpxdf.lat,'.', c='green', label="GPX points")
ax = fig.add_subplot(111)
ax.plot(gpxdf.lon,gpxdf.lat,'-', c='yellow', label="GPX line interp")
ax.legend()
fig.suptitle('GPX track for flight {}'.format(flight_id), fontsize=16)
fig.savefig(os.path.join(homedir, "{}_gpxtrack.png".format(os.path.splitext(logfile)[0])))
#fig.clear()

### Plot GPX on basemap – uses ipyleaflet and opens inline

In [ ]:
def df_to_linestring(df, lat='lat', lon='lon', z='ele'):
    """
    Turn a dataframe containing point data into a linestring in geojson format
    modified from: https://github.com/gboeing/urban-data-science/blob/3faf7e028d48cb03ddb999c5a910213c5384e7dc/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb
    
    df : the dataframe to convert to geojson
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # create a feature template to fill in
    feature = {'type':'Feature',
               'properties':{},
               'geometry':{'type':'LineString',
                           'coordinates':[]}}
    
    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # fill in the coordinates
        feature['geometry']['coordinates'].append([row[lon],row[lat],row[z]])

    # add this feature (aka, converted dataframe row) to the list of features inside our dict
    geojson['features'].append(feature)
    
    return(geojson)

In [ ]:
#TODO: figure out a way to cache the map layer or load a geotiff so we can do this offline
m = ipyl.Map(
    center=[np.mean([max(gpxdf.lat), min(gpxdf.lat)]), np.mean([max(gpxdf.lon), min(gpxdf.lon)])], 
    zoom=16, 
    layout=dict(width='600px', height='400px'), 
    basemap=ipyl.basemaps.Esri.WorldImagery)

def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)
    
dc = ipyl.DrawControl()
dc.on_draw(handle_draw)
m.add_control(dc)

# Plotting the points (even only the unique times) is unwieldy. Plotting all points crashed my kernel. Plotting the unique ones worked, but there's  no apparent way to customize the point display. 
flight_path = df_to_linestring(gpxdf)
m.add_layer(ipyl.GeoJSON(data=flight_path))
m

## Work on the photos

Quirks from ExifTool that required work-arounds: 

- ExifTool assumes that both the GPS and the image times are local unless another timezone is specified (unless taken from GPSTimeStamp which is UTC).

We normally set our camera to UTC and save the GPX files in UTC (by setting the computer time to UTC before running the Mission Planner conversion). Thus, running -geotag on a computer in local time without the '-geotime<${{DateTimeOriginal}}+00:00' part causes the camera times to be incorreclty adjusted to UTC. Our work-around uses the command to geotag images for which the camera clock was set to UTC (+00:00), using the time from DateTimeOriginal. It would also work to change the geosync value to account for this. 

In [ ]:
# Run the ExifTool command to geotag images with a GPX file
geosync = '-0:0:0'
# Geotag images for which the camera clock was set to UTC (+00:00), using the time from DateTimeOriginal: (from ExifTool docs)
cmd = """exiftool -v2 -geotag {} -geosync={} '-geotime<${{DateTimeOriginal}}+00:00' {}""".format(logfile, 
                                                                                                 geosync, imagefolder) 
subprocess.check_call(cmd, shell=True) 

I just moved the above cell up there and haven't tested the code in this order. 

In [ ]:
# List all JPEGS in imagefolder
flist=[os.path.join(imagefolder,f) for f in os.listdir(imagefolder) if f.lower().endswith('.jpg')]
print("Found {} images in {}.".format(len(flist),imagefolder))

# Set datetime formats
iso_fmt="%Y%m%dT%H%M%SZ"
tfmt_exif = '%Y:%m:%d %H:%M:%S' #e.g. 2017:05:04 14:14:12

In [ ]:
# Make dataframe of photos – get filename and DateTimeOriginal of each photo
dt = [datetime.datetime.strptime(Image.open(f)._getexif()[36867], tfmt_exif) for f in flist]
imgdf = pd.DataFrame({'orig_name': [os.path.basename(f) for f in flist],
                      'time_utc': dt,
                      'time_epoch': pd.to_datetime(dt, format = tfmt_exif).astype(np.int64) // 10**9, 
                      'time_iso': [t.strftime(iso_fmt) for t in dt],
                      'new_name': np.nan,
                      'lon': np.nan,
                      'lat': np.nan,
                      'ele': np.nan,
                      'interpolated': 0},
                        columns=['new_name', 'lat', 'lon', 'ele', 'time_utc', 'orig_name', 
                                 'time_epoch', 'time_iso', 'interpolated'])

imgdf

In [ ]:
# Export CSV for original photo EXIF times
imgcsvoutfile = imagefolder+'_imgtmp.csv'
imgdf.to_csv(imgcsvoutfile, index=False)
print ("Exported photo CSV file as:", imgcsvoutfile,"\n")
            
# print first and last image name and times
print("First file: {}, time: {}".format(imgdf.orig_name.iloc[0],imgdf.time_utc.iloc[0]))
print("Last file: {}, time: {}".format(imgdf.orig_name.iloc[-1],imgdf.time_utc.iloc[-1]),"\n")
# print first and last times in .gpx file
print("GPX data: {} from {} to {}".format(logfile, gpxdf.datetime_utc.iloc[0],gpxdf.datetime_utc.iloc[-1]))

In [ ]:
#%% Plot times of image vs GPX data
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(gpxdf.datetime_utc, gpxdf.ele,'.c', label='GPX')
ax.plot(imgdf.time_utc, np.tile(gpxdf.ele.max(), imgdf.shape[0]),'.r', label='photo')
ax.legend()
#fig.clear()

### Image Rename

In [ ]:
#TODO: don't run if the names have already been changed...
rename_photos = True

surveyid = fan.replace("-","")
if not os.path.exists(imgoutdir):
    shutil.copytree(imagefolder, imgoutdir)
for idx, row in imgdf.iterrows():
    img = row.orig_name
    namestr = "{}_{}{}_{}_{}".format(surveyid, flight_id, cam_id, row.time_iso, img)
    if rename_photos:
        os.rename(os.path.join(imagefolder, img), os.path.join(imgoutdir, namestr))
    imgdf.loc[idx, 'new_name'] = namestr

### Geotag photos & add standard USGS metadata to EXIF headers
I needed to hard code the location of exiftools - this might be different depending on ExifTools install. If needed, you can always run it at the command line flight-by-flight


In [ ]:
# # Run the ExifTool command to geotag images with a GPX file
# cmd = """exiftool -geosync=-0:0:0 -geotag {} {}""".format(logfile, imgoutdir)
# subprocess.check_call(cmd, shell=True)
# # Alternative?: subprocess.check_call("/usr/local/bin/exiftool -geosync=-0:0:0 -geotag {} {}""".format(logfile, imgoutdir)

In [ ]:
def write_WHSC_exiftags(imgdir, credit, comment, keywords, artist, contact):
    # Tags that will be identical for all images in the folder
    tagvalues = {}
    tagvalues['imgdir'] = imgdir
    tagvalues['credit'] = credit
    tagvalues['artist'] = artist
    tagvalues['contact'] = contact
    tagvalues['comment'] = comment
    tagvalues['keywords'] = keywords
    tagvalues['copyright'] = "Public Domain. Please credit {credit}".format(**tagvalues)
    # Write to EXIF
    cmd = """exiftool -Artist="{artist} " -Credit="{credit} " -Contact="{contact} " -comment="{comment} " -sep ", " -keywords="{keywords} " -Caption="{comment} " -Copyright="{copyright} " -CopyrightNotice="{copyright} " -Caption-Abstract="{comment} " -ImageDescription="{comment} " {imgdir}""".format(**tagvalues)
    subprocess.check_call(cmd, shell=True)
    print("Updated Exif headers in directory: {}".format(imgdir))
    return(True)

# Run ExifTool again to update other USGS specific meta tags
write_WHSC_exiftags(imgoutdir, credit, comment, keywords, artist, contact)